<a href="https://colab.research.google.com/github/vonnan/IBM-capstone-project/blob/main/IBM_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###IBM Capstone Project

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Webscraping from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To create the above dataframe:

1. The dataframe will consist of three columns: PostalCode, 
Borough, and Neighborhood
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.





### Use pd.read_html for webscraping the Toronto data

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood



In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df1, df2, df3 = pd.read_html(url)

df1.columns = ["PostalCode", "Borough", "Neighborhood"]




### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [ ]:
toronto = df1.loc[df1["Borough"] != "Not assigned"].copy() # remove the unassigned borough

toronto.sort_values("PostalCode", inplace = True)

###If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making. 

###In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



In [ ]:
toronto[toronto["Neighborhood"] == "Not assigned"]["Neighborhood"] = toronto[toronto["Neighborhood"] == "Not assigned"]["Borough"] #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

print("There are {} rows, i.e., postal codes in the dataframe".format(toronto.shape[0])) 

toronto.head()

There are 103 rows, i.e., postal codes in the dataframe


,PostalCode,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


### The above is for the first link submission, requirement 1-4

(Submit a link to your Notebook on your Github repository. (10 marks))

###we need to get the latitude and the longitude coordinates of each neighborhood.
First try use geocoder.google to get the latitude and longitude information

Step 1: Install geocoder and import it

In [ ]:
!pip install geocoder
import geocoder


In [ ]:
"""

def get_lat_long(postalcode):
  counter = 0
  location = None
  while (location == None):
    location = geocoder.google("{}, Toronto, Ontario".format(postalcode)).latlon
    counter += 1
    if counter ==10:
      break
  
  return location

toronto["location"] = toronto["PostalCode"].apply(get_lat_long)

toronto.head()
"""

###Geocoder.google is not reliable.
We have to user the webscraping to obtain the latitude/longitude for the Toronto borough/neighborhood

In [ ]:
url = "https://cocl.us/Geospatial_data"
PC_location = pd.read_csv(url)
PC_location.rename(columns = {"Postal Code": "PostalCode"}, inplace = True)
toronto = toronto.merge(PC_location, how = "left", on= "PostalCode")


In [ ]:
print(toronto.shape[0])
toronto.head(12)

103


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Above is the second submission for the dataframe created with the latitude/longitude 

Below we are going to do the last part of the 3rd week assignment

In [ ]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library




### We will focus on the 4 boroughs that contain "Toronto" in the Borough name; create a new dataframe of Toronto_city

In [182]:
toronto_city = toronto[toronto["Borough"].str.contains("Toronto")].copy().reset_index(drop=True)
toronto_city.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [ ]:
boroughs = list(toronto_city["Borough"].unique())
colors =["red", "blue", "green", "purple"]


###Draw the map of Toronto

Step 1: get the latitude/longitude of Toronto

In [ ]:
address = "Toronto, Ontario"

toronto_map = folium.Map

location = Nominatim(user_agent="toronto_explorer").geocode(address)

lat, lng = location.latitude, location.longitude
lat,lng

(43.6534817, -79.3839347)

Step 2: Draw the map of Toronto using Folium.Map

In [ ]:
toronto_map = folium.Map([lat, lng], zoom_start=12)

for lat, lng, brgh, hood in zip(toronto_city["Latitude"], toronto_city["Longitude"], toronto_city["Borough"], toronto_city["Neighborhood"]):
  label = brgh + ": " + hood
  for i in range(4):
    if brgh == boroughs[i]:
      color = colors[i]
      
  folium.CircleMarker(
      [lat, lng],
      radius = 10,
      popup = label,
      fill = True,
      color = color,
      fill_color = color,
      fill_opacity = 0.3
  ).add_to(toronto_map)

In [ ]:
toronto_map

###First, just use Kmeans clustering to divide the clustering based on latitude and longitude 

In [ ]:
KM = KMeans(n_clusters= len(boroughs), n_init= 12, random_state=0)
X = toronto_city[["Latitude", "Longitude"]]
KM.fit(X)
colors2 = []
KM.labels_
for label in KM.labels_:
  if label == 1:
    color = "red"
  elif label == 2:
    color = "blue"
  elif label == 3:
    color = "green"
  else:
    color = "purple"
  colors2.append(color)

for lat, lng, brgh, hood, color in zip(toronto_city["Latitude"], toronto_city["Longitude"], toronto_city["Borough"], toronto_city["Neighborhood"], colors2):
  label = brgh + ": " + hood
  folium.CircleMarker(
      [lat, lng],
      radius = 4,
      popup = label,
      fill = True,
      color = color,
      fill_color = color,
      fill_opacity = 0.3
  ).add_to(toronto_map)

### KMeans is actually doing a good job. The prediction is correct 37 out of 39 PostalCodes. 

### Two wrong: Downtown Toronto: Christie is clustered to the rest of West Toronto; Central Toronto: The Annex, North Midtown, Yorkville is clustered to Downtown Toronto

In [ ]:
toronto_map

###We are going to collect all the venues from the FourSquare
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

In [ ]:
CLIENT_ID = 'KWTBOAQUOFKZSP5QHAZ530MZ5WKAUPBPCFZKQJNVEM5EJV1R' # your Foursquare ID
CLIENT_SECRET = 'LP251NSOTLWM5PWMYKWNCRMMFSSRRZC5D4PHSGLASZ1GXK5T' # your Foursquare Secret
ACCESS_TOKEN = '2NX33SVZLUCMUQOF01CKEZ2PHKA3VAR4NPDCGD4DWOMNVTHP' # your FourSquare Access Token
VERSION = '20201204'
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: KWTBOAQUOFKZSP5QHAZ530MZ5WKAUPBPCFZKQJNVEM5EJV1R
CLIENT_SECRET: LP251NSOTLWM5PWMYKWNCRMMFSSRRZC5D4PHSGLASZ1GXK5T


In [ ]:
def getNearByVenues(names, lats, lngs, radius = 500):
  venue_list = []
  for name, lat, lng in zip(names, lats, lngs):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format( 
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    results = requests.get(url).json()["response"]["groups"][0]["items"]
    venue_list.append([(
        name,
        lat,
        lng,
        v["venue"]["name"],
        v["venue"]["location"]["lat"],
        v["venue"]["location"]["lng"],
        v["venue"]["categories"][0]["name"]) for v in results])

  venue_df = pd.DataFrame([item for v in venue_list for item in v])

  venue_df.columns =[
                      "Neighborhood",
                      "Neighborhood Latitude",
                      "Neighborhood Longitude",
                      "Venue Name",
                      "Venue Latitude",
                      "Venue Longitude",
                      "Venue Category"
  ]

  return venue_df



In [ ]:
toronto_city_venues = getNearByVenues(toronto_city["Neighborhood"], toronto_city["Latitude"], toronto_city["Longitude"], radius = 500)
toronto_city_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [ ]:
toronto_onehot = pd.DataFrame()
toronto_onehot["Neighborhoods"] = toronto_city_venues["Neighborhood"]
dummy_df = pd.get_dummies(toronto_city_venues[["Venue Category"]], prefix= "", prefix_sep="")
toronto_onehot = pd.concat([toronto_onehot, dummy_df], axis = 1)
toronto_onehot.head()

,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
toronto_grouped = toronto_onehot.groupby("Neighborhoods").mean()
KM = KMeans(n_clusters = 4, n_init=12, random_state=0)
KM.fit(toronto_grouped)
KM.labels_

colors3=[]
for label in KM.labels_:
  if label == 1:
    color = "red"
  elif label == 2:
    color = "blue"
  elif label == 3:
    color = "green"
  else:
    color = "purple"
  colors3.append(color)

for lat, lng, brgh, hood, color in zip(toronto_city["Latitude"], toronto_city["Longitude"], toronto_city["Borough"], toronto_city["Neighborhood"], colors3):
  label = brgh + ": " + hood
  folium.CircleMarker(
      [lat, lng],
      radius = 16,
      popup = label,
      fill = True,
      color = color,
      fill_color = color,
      fill_opacity = 0.2
  ).add_to(toronto_map)


In [170]:
toronto_map

In [139]:
toronto_grouped.head()

number_top_venues = 10
def get_top_venues(row, number_top_venues):
  return row.sort_values(ascending = False).head(number_top_venues).index.values
     

###Generate top venue data frame

In [181]:
ind = ["st", "nd", "rd"] #1st, 2nd, 3rd
columns = ["Neighborhood"]
for i in range(number_top_venues):
  if i<=2:
    columns.append("{}{} Most Popular Venue".format(i+1, ind[i]))
  else:
    columns.append("{}th Most Popular Venue".format(i+1))

toronto_top_venue = pd.DataFrame(columns=columns)
toronto_top_venue["Neighborhood"] =toronto_grouped.index.values
for i in range(len(toronto_grouped)):
  row = toronto_grouped.iloc[i,:].T
  toronto_top_venue.iloc[i, 1:] = get_top_venues(row, number_top_venues)
toronto_top_venue["label"] =KM.labels_
cols = [toronto_top_venue.columns[-1]] + list(toronto_top_venue.columns[:-1])
toronto_top_venue = toronto_top_venue[cols]
toronto_top_venue.sort_values("label", inplace=True)
toronto_top_venue

,label,Neighborhood,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
35,0,The Beaches,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
0,1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Cheese Shop,Seafood Restaurant,Restaurant,Farmers Market,Sandwich Place,Breakfast Spot
21,1,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
22,1,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Dessert Shop,Cuban Restaurant,Eastern European Restaurant,Dog Run,Italian Restaurant,Bar,Restaurant,Movie Theater
23,1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop
24,1,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Café,Theater,Breakfast Spot,Event Space,Shoe Store,Hotel
25,1,"Richmond, Adelaide, King",Coffee Shop,Café,Gym,Restaurant,Hotel,Thai Restaurant,Bar,Clothing Store,Cosmetics Shop,Concert Hall
28,1,"Runnymede, Swansea",Café,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Sushi Restaurant,Yoga Studio,Gourmet Shop,Restaurant,Latin American Restaurant
29,1,St. James Town,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant
30,1,"St. James Town, Cabbagetown",Coffee Shop,Pizza Place,Restaurant,Café,Italian Restaurant,Market,Chinese Restaurant,Bakery,Park,Pub


###Merge two data frames so that we will have both geolocations and most popular venue information in the same dataframe

In [183]:
toronto_city =toronto_city.merge(right = toronto_top_venue, on = "Neighborhood", how = "inner")
toronto_city

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Sushi Restaurant,Indian Restaurant,Spa
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Gym,Sushi Restaurant,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Pub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Diner,Bar,Ice Cream Shop,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Colombian Restaurant,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Department Store,Breakfast Spot,Gym / Fitness Center,Donut Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Restaurant,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Trail,Playground,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,1,Coffee Shop,Restaurant,American Restaurant,Bank,Supermarket,Sushi Restaurant,Fried Chicken Joint,Pizza Place,Bagel Shop,Pub


In [185]:
toronto_city.sort_values("label", inplace = True)
toronto_city

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Gym,Sushi Restaurant,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Pub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Diner,Bar,Ice Cream Shop,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Colombian Restaurant,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Department Store,Breakfast Spot,Gym / Fitness Center,Donut Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Restaurant,Seafood Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Sushi Restaurant,Indian Restaurant,Spa
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Department Store,Thai Restaurant
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Pizza Place,Restaurant,Café,Italian Restaurant,Market,Chinese Restaurant,Bakery,Park,Pub


In [192]:
toronto_map

###Cluster Analysis

### Cluster 0: the beach: 
Top venues are pub, trail, health Food store, Neighborhood, Yoga studio, Dog Run etc. It is the beach. Relax and healthy


In [186]:
toronto_city[toronto_city["label"] == 0]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Neighborhood,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


###Cluster 2: Roselawn Neighborhood

Music venue, Garden, Yoga Studio, Department Store, Escape Room etc...This neighborhood is for entertainment

In [187]:
toronto_city[toronto_city["label"] == 2]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Music Venue,Garden,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 3:
Forest Hill North & West, Forest Hill Road Park, 
Moore Park, Summerhill East, 
Rosedale

Top venues are parks or trails, playground, Yoga studio. It is a outdoor park/trail neighborhoods

In [190]:
print(toronto_city[toronto_city["label"] == 3]["Neighborhood"].tolist())
toronto_city[toronto_city["label"] == 3]

['Forest Hill North & West, Forest Hill Road Park', 'Moore Park, Summerhill East', 'Rosedale']


,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
23,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,3,Park,Sushi Restaurant,Jewelry Store,Trail,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Trail,Playground,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,3,Park,Trail,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


###Cluster 1:
Rest of toronto city
Top venues are coffee shops, cafe, restaurant, pizza place, etc

In [191]:
print(toronto_city[toronto_city["label"] == 1]["Neighborhood"].tolist())
toronto_city[toronto_city["label"] == 1]

['India Bazaar, The Beaches West', 'Studio District', 'Lawrence Park', 'Davisville North', 'North Toronto West, Lawrence Park', 'Davisville', 'The Danforth West, Riverdale', 'Central Bay Street', 'St. James Town, Cabbagetown', 'Church and Wellesley', 'Regent Park, Harbourfront', 'Garden District, Ryerson', 'St. James Town', 'Business reply mail Processing Centre, South Central Letter Processing Plant Toronto', 'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park', 'Harbourfront East, Union Station, Toronto Islands', 'Berczy Park', "Queen's Park, Ontario Provincial Government", 'Toronto Dominion Centre, Design Exchange', 'Commerce Court, Victoria Hotel', 'The Annex, North Midtown, Yorkville', 'University of Toronto, Harbord', 'Kensington Market, Chinatown, Grange Park', 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport', 'Richmond, Adelaide, King', 'First Canadian Place, Underground city', 'Stn A PO Boxes', 'Dufferi

,PostalCode,Borough,Neighborhood,Latitude,Longitude,label,1st Most Popular Venue,2nd Most Popular Venue,3rd Most Popular Venue,4th Most Popular Venue,5th Most Popular Venue,6th Most Popular Venue,7th Most Popular Venue,8th Most Popular Venue,9th Most Popular Venue,10th Most Popular Venue
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Gym,Sushi Restaurant,Brewery,Liquor Store,Restaurant,Burrito Place,Italian Restaurant,Fast Food Restaurant,Pub
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Diner,Bar,Ice Cream Shop,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Bus Line,Swim School,Colombian Restaurant,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Park,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Department Store,Breakfast Spot,Gym / Fitness Center,Donut Shop
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Sandwich Place,Dessert Shop,Pizza Place,Gym,Sushi Restaurant,Italian Restaurant,Coffee Shop,Café,Restaurant,Seafood Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Sushi Restaurant,Indian Restaurant,Spa
17,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Department Store,Thai Restaurant
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Pizza Place,Restaurant,Café,Italian Restaurant,Market,Chinese Restaurant,Bakery,Park,Pub
12,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
